<a href="https://colab.research.google.com/github/jinseonggram/Baseline-sentiment-classification/blob/master/baseline_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import requirements

In [1]:
!pip install Sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.0 MB/s 
     |████████████████████████████████| 163 kB 57.7 MB/s 


In [2]:
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict

import torch
from torch.nn.utils.rnn import pad_sequence

import numpy as np
from tqdm import tqdm, trange

from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW,
    
    AlbertTokenizer, AlbertForSequenceClassification
)

# 1. Preprocess

In [3]:
def make_id_file(task, tokenizer):
    def make_data_strings(file_name):
        data_strings = []
        with open(os.path.join(file_name), 'r', encoding='utf-8') as f:
            id_file_data = [tokenizer.encode(line.lower()) for line in f.readlines()]
        for item in id_file_data:
            data_strings.append(' '.join([str(k) for k in item]))
        return data_strings
    
    print('it will take some times...')
    train_pos = make_data_strings('sentiment.train.1')
    train_neg = make_data_strings('sentiment.train.0')
    dev_pos = make_data_strings('sentiment.dev.1')
    dev_neg = make_data_strings('sentiment.dev.0')

    print('make id file finished!')
    return train_pos, train_neg, dev_pos, dev_neg

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
from google.colab import files
uploaded = files.upload()

Saving sentiment.train.0 to sentiment.train.0
Saving sentiment.train.1 to sentiment.train.1
Saving test_no_label.csv to test_no_label.csv
Saving sentiment.dev.1 to sentiment.dev.1
Saving sentiment.dev.0 to sentiment.dev.0


In [6]:
!ls

sample_data	 sentiment.dev.1    sentiment.train.1
sentiment.dev.0  sentiment.train.0  test_no_label.csv


In [7]:
train_pos, train_neg, dev_pos, dev_neg = make_id_file('yelp', tokenizer)

it will take some times...
make id file finished!


In [8]:
train_pos[:10]

['101 6581 2833 1012 102',
 '101 21688 8013 2326 1012 102',
 '101 2027 2036 2031 3679 19247 1998 3256 6949 2029 2003 2428 2204 1012 102',
 '101 2009 1005 1055 1037 2204 15174 2098 7570 22974 2063 1012 102',
 '101 1996 3095 2003 5379 1012 102',
 '101 2204 3347 2833 1012 102',
 '101 2204 2326 1012 102',
 '101 11350 1997 2154 2003 25628 1998 7167 1997 19247 1012 102',
 '101 2307 2173 2005 6265 2030 3347 27962 1998 5404 1012 102',
 '101 1996 2047 2846 3504 6429 1012 102']

In [9]:
class SentimentDataset(object):
    def __init__(self, tokenizer, pos, neg):
        self.tokenizer = tokenizer
        self.data = []
        self.label = []

        for pos_sent in pos:
            self.data += [self._cast_to_int(pos_sent.strip().split())]
            self.label += [[1]]
            # print('pos_sent: ', self.label, pos_sent)
        for neg_sent in neg:
            self.data += [self._cast_to_int(neg_sent.strip().split())]
            self.label += [[0]]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

In [10]:
train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)

In [11]:
# for i, item in reversed(list(enumerate(train_dataset))):
#     print(item)
#     if i == 1:
#         break

for i, item in enumerate(train_dataset):
    print(item)
    if i == 1:
        break

(array([ 101, 6581, 2833, 1012,  102]), array([1]))
(array([  101, 21688,  8013,  2326,  1012,   102]), array([1]))


In [12]:
# def collate_fn_style(samples):
#     # print('samples : ',samples)
#     input_ids, labels = zip(*samples)
#     # print('inputs : ', input_ids)
#     # print('labels : ', labels)

#     # batch 사이즈중 가장 긴 문장의 길이 추출
#     max_len = max(len(input_id) for input_id in input_ids)

#     # 조금 더 잘 pad 를 하기 위해 문장 정리
#     sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    
#     # 위에 오류
#     # sorted_indices = range(len(input_ids))

#     # https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html, 길이를 맞추기 위해 pad 를 해줌. 전체적인 dim 이 같아짐.
#     input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
#                              batch_first=True)
    
#     # 불필요한 영역에는 attention 을 두지 않도록 attention mask 생성.
#     attention_mask = torch.tensor(
#         [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
#          sorted_indices])

#     # attention_mask = []
#     # for seq in input_ids:
#     #   seq_mask = [float(i>0) for i in seq]
#     #   attention_mask.append(seq_mask)
    
#     # attention_mask = torch.tensor(attention_mask)
      
#     token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
#     position_ids = torch.tensor([list(range(len(input_ids[index]))) for index  in sorted_indices])
#     labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

#     return input_ids, attention_mask, token_type_ids, position_ids, labels


def collate_fn_style(samples):
    input_ids, labels = zip(*samples)
    max_len = max(len(input_id) for input_id in input_ids)
    # sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1] 
    sorted_indices = range(len(input_ids))

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])
    # labels = torch.tensor(labels)
    

    return input_ids, attention_mask, token_type_ids, position_ids, labels

In [13]:
train_batch_size=128 # 32-> 128
eval_batch_size=256 # 64-> 256

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

In [14]:
for idx, i in enumerate(train_loader):
    input_ids, attention_mask, token_type_ids, position_ids, labels = i
    print(i)
    if idx == 0:
      break
    # print(i)
# train_loader.dataset[0]

[tensor([[  101,  2023,  2003,  ...,     0,     0,     0],
        [  101,  1045,  2018,  ...,     0,     0,     0],
        [  101,  2028,  2154,  ...,     0,     0,     0],
        ...,
        [  101,  1996, 15890,  ...,   102,     0,     0],
        [  101,  3294,  9951,  ...,     0,     0,     0],
        [  101,  2026,  2767,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), tensor([[ 0,  1,  2,  ..., 17, 18, 19],
        [ 0,  1,  2,  ..., 17, 18, 19],
        [ 0,  1,  2,  ..., 17, 18, 19],
        ...,
        [ 0,  1,  2,  ..., 17, 18, 19],
        [ 0,  1,  2,  ...

In [15]:
# random seed
random_seed=33 # 42 -> 33
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = AlbertForSequenceClassification.from_pretrained('albert-base-v2')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# GPU 에 얹어주는 작업
model.to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
model.train()
learning_rate = 5e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [17]:
def compute_acc(predictions, target_labels):
    return (np.array(predictions) == np.array(target_labels)).mean()

In [18]:
train_epoch = 3
lowest_valid_loss = 9999.

for epoch in range(train_epoch):

    # tqdm 은 Bar 형태로 학습량을 알려주는 module.
    with tqdm(train_loader, unit="batch") as tepoch:


        for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")

            # 모든 데이터를 GPU 로 옮겨줌.
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            position_ids = position_ids.to(device)
            labels = labels.to(device, dtype=torch.long)

            optimizer.zero_grad()

            # forward-propagation 연산 진행
            output = model(input_ids=input_ids,
                           attention_mask=attention_mask,
                           token_type_ids=token_type_ids,
                           position_ids=position_ids,
                           labels=labels)

            loss = output.loss

            # back-propagation 연산
            loss.backward()

            # 가중치 update
            optimizer.step()  

            # tqdm 에서 loss 를 출력하기 위해 loss 를 넣어줌.
            tepoch.set_postfix(loss=loss.item())


            # iteration : trainloader 의 전체 개수.
            # int(len(train_loader) / 5) == 0 마다 validation set 으로 evaluate.
            if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
                # Evaluate the model five times per epoch
                with torch.no_grad():
                    model.eval()
                    valid_losses = []
                    predictions = []
                    target_labels = []
                    for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                desc='Eval',
                                                                                                position=1,
                                                                                                leave=None):
                        input_ids = input_ids.to(device)
                        attention_mask = attention_mask.to(device)
                        token_type_ids = token_type_ids.to(device)
                        position_ids = position_ids.to(device)
                        labels = labels.to(device, dtype=torch.long)

                        output = model(input_ids=input_ids,
                                       attention_mask=attention_mask,
                                       token_type_ids=token_type_ids,
                                       position_ids=position_ids,
                                       labels=labels)

                        logits = output.logits
                        loss = output.loss
                        valid_losses.append(loss.item())

                        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                        batch_labels = [int(example) for example in labels]

                        predictions += batch_predictions
                        target_labels += batch_labels

                acc = compute_acc(predictions, target_labels)
                valid_loss = sum(valid_losses) / len(valid_losses)
                if lowest_valid_loss > valid_loss:
                    print('Acc for model which have lower valid loss: ', acc)
                    torch.save(model.state_dict(), "./pytorch_model.bin")

Eval: 100%|██████████| 16/16 [00:02<00:00,  5.81it/s]
                                                     

Acc for model which have lower valid loss:  0.9755


Eval: 100%|██████████| 16/16 [00:02<00:00,  5.90it/s]
                                                     

Acc for model which have lower valid loss:  0.97925


Eval: 100%|██████████| 16/16 [00:02<00:00,  5.87it/s]
                                                     

...

Eval: 100%|██████████| 16/16 [00:02<00:00,  5.86it/s]
                                                     

Acc for model which have lower valid loss:  0.97875


Eval: 100%|██████████| 16/16 [00:02<00:00,  5.91it/s]
                                                     

Acc for model which have lower valid loss:  0.97825


Epoch 2: 100%|██████████| 3463/3463 [16:19<00:00,  3.54batch/s, loss=0.00434]


In [19]:
import pandas as pd
test_df = pd.read_csv('test_no_label.csv')

In [20]:
test_dataset = test_df['Id']

In [21]:
def make_id_file_test(tokenizer, test_dataset):
    data_strings = []
    id_file_data = [tokenizer.encode(sent.lower()) for sent in test_dataset]
    for item in id_file_data:
        data_strings.append(' '.join([str(k) for k in item]))
    return data_strings

In [22]:
test = make_id_file_test(tokenizer, test_dataset)

In [23]:
test[:10]

['101 2009 1005 1055 1037 2878 2047 3325 1998 2047 26389 2169 2051 2017 2175 1012 102',
 '101 2061 15640 2013 2019 2214 5440 1012 102',
 '101 2009 2003 1996 2087 14469 7273 1999 1996 3028 1012 102',
 '101 2079 2025 3696 1037 10084 2007 2122 2111 1012 102',
 '101 1045 2001 6091 1998 2016 2081 2033 2514 2061 6625 1998 6160 1012 102',
 '101 1996 2069 2518 2057 2363 2008 2001 2980 2001 1996 4157 1012 102',
 '101 2053 1010 2025 1996 3924 2012 2004 2226 1010 1996 3924 1999 3502 2152 1012 102',
 '101 2027 3288 2009 2041 2392 2005 2017 1998 2024 2200 14044 1012 102',
 '101 4606 1996 12043 2106 1050 1005 1056 2130 2113 2129 2000 2147 1996 3274 1012 102',
 '101 2027 2031 2019 6581 4989 1997 25025 2015 2000 5454 2013 1012 102']

In [24]:
class SentimentTestDataset(object):
    def __init__(self, tokenizer, test):
        self.tokenizer = tokenizer
        self.data = []

        for sent in test:
            self.data += [self._cast_to_int(sent.strip().split())]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample)

In [25]:
test_dataset = SentimentTestDataset(tokenizer, test)

In [26]:
def collate_fn_style_test(samples):
    input_ids = samples
    max_len = max(len(input_id) for input_id in input_ids)

    # 버그.
    # smaples 가 input id 만 가지고 있음. trainset 은 input_ids 와 labels 둘 다 가졌음. test_dataset 은 labels 이 없음.
    # input 의 순서를 바꿔주면 kaggle 에 내가 모르는 labels set 과 순서가 얻 바뀌게 된다.
    # sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    sorted_indices = range(len(input_ids))

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids

In [27]:
test_batch_size = 32
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [28]:
with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)

        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions += batch_predictions


Test:  91%|█████████ | 29/32 [00:00<00:00, 33.95it/s]
                                                     

In [29]:
test_df['Category'] = predictions

In [30]:
test_df.to_csv('submission.csv', index=False)

# 새 섹션